# Project imports

In [ ]:
"""
All needed imports included here
"""

# Data Loading step

In [ ]:
"""
Create data loaders and augmentations needed here
"""

# Reconstruction Networks

In [ ]:
"""
creation, training, and testing of the image2mesh reconstruction networks
"""

# Purifying predicted Meshes

In [ ]:
"""
Code to purify meshes predicted by the previous networks to be used in the retrieval step
"""

# Mesh Encoding

In [ ]:
"""
AutoEncoder Models and/or different techniques used to encode the mesh to a smaller dimensions
"""

# Mesh Retreival Networks

In [ ]:
"""
Models/Techniques to use the previous encoding steps to retreive objects from a specified database
"""

# Inference and Full Testing

In [ ]:
"""
Testing the entire pipeline implemented with added visualizations and discussions.
"""

# Citations

[1].....